In [8]:
import glob
import pandas as pd
import polars as pl

# Define the file pattern
file_pattern = '/Users/famadeo/Development/Blockhouse/XNAS-20250105-SEG874YYUF/xnas-itch-202412??.mbp-10.csv'

# Get the list of files matching the pattern
file_list = glob.glob(file_pattern)

# Load each file into a DataFrame and store them in a list
dataframes = [pl.read_csv(file) for file in file_list]

# Optionally, concatenate all DataFrames into a single DataFrame
df = pl.concat(dataframes, how="vertical_relaxed")

# Display the combined DataFrame
df.head()

ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_ct_02,ask_ct_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_ct_03,ask_ct_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_ct_04,ask_ct_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_ct_05,ask_ct_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_ct_06,ask_ct_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_ct_07,ask_ct_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
str,str,i64,i64,i64,str,str,i64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,str,f64,i64,i64,i64,i64,str,f64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,str
"""2024-12-12T09:00:00.003098865Z""","""2024-12-12T09:00:00.002932488Z""",10,2,780,"""A""","""N""",0,270.98,25,130,166377,283354,270.98,null,25,0,1,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""AMGN"""
"""2024-12-12T09:00:00.003413711Z""","""2024-12-12T09:00:00.003247133Z""",10,2,780,"""A""","""A""",0,276.08,25,130,166578,283365,270.98,276.08,25,25,1,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""AMGN"""
"""2024-12-12T09:00:00.013663864Z""","""2024-12-12T09:00:00.013497510Z""",10,2,16244,"""A""","""N""",0,427.0,680,130,166354,283688,null,427.0,0,680,0,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""TSLA"""
"""2024-12-12T09:00:00.013746435Z""","""2024-12-12T09:00:00.013577994Z""",10,2,16244,"""A""","""A""",1,428.0,800,130,168441,283694,null,427.0,0,680,0,1,null,428.0,0,800,0,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""TSLA"""
"""2024-12-12T09:00:00.014044826Z""","""2024-12-12T09:00:00.013878689Z""",10,2,38,"""A""","""N""",0,258.9,41,130,166137,283706,null,258.9,0,41,0,1,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,null,null,0,0,0,0,"""AAPL"""


In [9]:
# Get the unique symbols from the orderbook
symbols = df['symbol'].unique()

# Create a dictionary to store DataFrames for each symbol
orderbook_dict = {symbol: df.filter(pl.col('symbol') == symbol) for symbol in symbols}

# Display the dictionary keys to verify
print(orderbook_dict.keys())

dict_keys(['AAPL', 'AMGN', 'TSLA', 'XOM', 'JPM'])


In [16]:
# Filter columns to include only numeric types
numeric_columns = [col for col in df.columns if df[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]]

# Calculate NaN counts for numeric columns
nan_counts = df.select([pl.col(col).is_nan().sum().alias(col) for col in numeric_columns])
print(nan_counts)


shape: (1, 57)
┌───────┬──────────────┬───────────────┬───────┬───┬───────────┬───────────┬───────────┬───────────┐
│ rtype ┆ publisher_id ┆ instrument_id ┆ depth ┆ … ┆ bid_sz_09 ┆ ask_sz_09 ┆ bid_ct_09 ┆ ask_ct_09 │
│ ---   ┆ ---          ┆ ---           ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ u32   ┆ u32          ┆ u32           ┆ u32   ┆   ┆ u32       ┆ u32       ┆ u32       ┆ u32       │
╞═══════╪══════════════╪═══════════════╪═══════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0     ┆ 0            ┆ 0             ┆ 0     ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ 0         │
└───────┴──────────────┴───────────────┴───────┴───┴───────────┴───────────┴───────────┴───────────┘
shape: (1, 74)
┌─────────┬──────────┬───────┬──────────────┬───┬───────────┬───────────┬───────────┬────────┐
│ ts_recv ┆ ts_event ┆ rtype ┆ publisher_id ┆ … ┆ ask_sz_09 ┆ bid_ct_09 ┆ ask_ct_09 ┆ symbol │
│ ---     ┆ ---      ┆ ---   ┆ ---          ┆   ┆ ---       ┆ ---       ┆

In [ ]:
null_counts = df.select([pl.col(col).is_null().sum().alias(col) for col in df.columns])
print(null_counts)

In [17]:
# Save each Polars DataFrame to a Parquet file
for symbol, orderbook in orderbook_dict.items():
    filename = f"{symbol}_20241202-20250104.parquet"
    orderbook = orderbook.fill_null(strategy="forward").fill_null(strategy="backward")
    orderbook.write_parquet(filename)
    print(f"Saved {symbol} to {filename}")

Saved AAPL to AAPL_20241202-20250104.parquet
Saved AMGN to AMGN_20241202-20250104.parquet
Saved TSLA to TSLA_20241202-20250104.parquet
Saved XOM to XOM_20241202-20250104.parquet
Saved JPM to JPM_20241202-20250104.parquet


In [12]:
for symbol, orderbook in orderbook_dict.items():
    print(f"{symbol} -- {orderbook.shape}")


AAPL -- (24850883, 74)
AMGN -- (1550200, 74)
TSLA -- (38744363, 74)
XOM -- (8239659, 74)
JPM -- (3165655, 74)


In [ ]:
tesla_orderbook = orderbook_df[orderbook_df['Symbol'] == 'TSLA']

In [23]:
import os
import yaml
import polars as pl

def load_symbol_data(symbol_name):
    # Load the configuration file
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    
    # Check if the symbol name exists in the config
    if symbol_name not in config:
        raise ValueError(f"Symbol '{symbol_name}' not found in config.yaml")
    
    # Get the file path for the symbol
    filename = config[symbol_name]
    
    # Check if the file path is valid
    if not  filename or not isinstance(filename, str):
        raise ValueError(f"Invalid file path for symbol '{symbol_name}' in config.yaml")
    
    # Load the parquet file into a Polars DataFrame
    try:
        current_dir = os.getcwd()
        # Construct the relative path to the file
        filename = os.path.join(current_dir, '..', 'data', filename)

        df = pl.read_parquet(filename)
    except Exception as e:
        raise IOError(f"Error loading file '{filename}': {e}")
    
    return df

# Example usage
df = load_symbol_data('AAPL')
print(df.head())

shape: (5, 74)
┌─────────────┬─────────────┬───────┬─────────────┬───┬───────────┬───────────┬───────────┬────────┐
│ ts_recv     ┆ ts_event    ┆ rtype ┆ publisher_i ┆ … ┆ ask_sz_09 ┆ bid_ct_09 ┆ ask_ct_09 ┆ symbol │
│ ---         ┆ ---         ┆ ---   ┆ d           ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---    │
│ str         ┆ str         ┆ i64   ┆ ---         ┆   ┆ i64       ┆ i64       ┆ i64       ┆ str    │
│             ┆             ┆       ┆ i64         ┆   ┆           ┆           ┆           ┆        │
╞═════════════╪═════════════╪═══════╪═════════════╪═══╪═══════════╪═══════════╪═══════════╪════════╡
│ 2024-12-12T ┆ 2024-12-12T ┆ 10    ┆ 2           ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ AAPL   │
│ 09:00:00.01 ┆ 09:00:00.01 ┆       ┆             ┆   ┆           ┆           ┆           ┆        │
│ 4044826Z    ┆ 3878689Z    ┆       ┆             ┆   ┆           ┆           ┆           ┆        │
│ 2024-12-12T ┆ 2024-12-12T ┆ 10    ┆ 2           ┆ … ┆ 0         ┆ 0       